In [ ]:
%cd sample_data

/content/sample_data


In [ ]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('TwitterDataset.csv').rename(columns={"Label": "label"})
print(np.shape(dataset))


(3006, 2)


In [ ]:
dataset[:2]

,Tweet,label
0,احساس می‌کنم غایت زندگی من اینه که یکی یروز بف...,4
1,ای بابا نصیب هرکسی نمیشه که، فقط دعای خیر پدر ...,0


In [ ]:
# train, test = train_test_split(df, test_size=0.15)
# print(np.shape(train), np.shape(test))
from datasets import Dataset
dataset = Dataset.from_pandas(dataset)

In [ ]:
dataset[:2]

{'Tweet': ['احساس می\u200cکنم غایت زندگی من اینه که یکی یروز بفهمه دارم راجع به چی حرف می\u200cزنم',
  'ای بابا نصیب هرکسی نمیشه که، فقط دعای خیر پدر مادر :))))))))'],
 'label': [4, 0]}

In [ ]:
# tweets = dataset.loc[:,"Tweet"]
# labels = dataset.loc[:,"Label"]

In [ ]:
dataset = dataset.train_test_split(test_size=0.15)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-base-uncased-sentiment-deepsentipers-multi")

def tokenize_function(examples):
    return tokenizer(examples["Tweet"], padding="max_length", truncation=True, max_length=128, return_tensors="pt")

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/2555 [00:00<?, ? examples/s]

Map:   0%|          | 0/451 [00:00<?, ? examples/s]

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [ ]:
# tweets = train.loc[:,"Tweet"]
# labels = train.loc[:,"Label"]

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import TransformerEncoder, TransformerEncoderLayer

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification  # for pytorch
from transformers import pipeline

from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("HooshvareLab/bert-fa-base-uncased-sentiment-deepsentipers-multi", num_labels=5)
model.config.id2label = {0: 'happy', 1: 'sad', 2: 'angry', 3: 'neutral', 4: 'emotional'}


In [ ]:
# for param in model.parameters():
#     param.requires_grad = False

# for param in model.classifier.parameters():
#     param.requires_grad = True

In [ ]:
!pip install evaluate datasets transformers==4.28.0


In [ ]:
small_train_dataset[0:1]

In [ ]:
small_eval_dataset[0:1]

{'Tweet': ['می\u200cدونستید چرا واحد زمانی ما هفته\u200cاس؟'],
 'Label': [3],
 'input_ids': [[2, 27000, 51302, 2756, 2898, 3015, 2179, 2759, 2070, 303, 4]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
import numpy as np
import evaluate
import torch.nn as nn

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    learning_rate=2e-4, num_train_epochs=15)

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 1.0, 3.0, 3.0, 3.0], device=model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.154605,0.538803
2,1.151300,1.220219,0.576497
3,1.151300,1.897134,0.541020


TrainOutput(global_step=960, training_loss=0.7981099446614583, metrics={'train_runtime': 229.4432, 'train_samples_per_second': 33.407, 'train_steps_per_second': 4.184, 'total_flos': 504200140496640.0, 'train_loss': 0.7981099446614583, 'epoch': 3.0})

In [ ]:
import pickle
with open("embeddings", "wb") as fp:   #Pickling
    pickle.dump(tweets_embedding, fp)


In [ ]:
import pickle
with open("embeddings", "rb") as fp:   # Unpickling
    tweets_embedding = pickle.load(fp)

In [ ]:
pipe = pipeline("text-classification", model=trainer.model, tokenizer=tokenizer, device='cpu')
pipe(["سگ توی روحت",
      "وقتی میبینمت حالم بد میشه",
      "خیلی خوشحالم"])